In [7]:
from itertools import combinations

def unify(x, y, theta={}):
    """Unify two predicates using a substitution (theta)."""
    if theta is None:
        return None
    elif x == y:
        return theta
    elif isinstance(x, str) and x.islower():
        return unify_var(x, y, theta)
    elif isinstance(y, str) and y.islower():
        return unify_var(y, x, theta)
    elif isinstance(x, tuple) and isinstance(y, tuple) and len(x) == len(y):
        return unify(x[1:], y[1:], unify(x[0], y[0], theta))
    else:
        return None

def unify_var(var, x, theta):
    """Unify variable."""
    if var in theta:
        return unify(theta[var], x, theta)
    elif x in theta:
        return unify(var, theta[x], theta)
    else:
        theta[var] = x
        return theta

def resolve(clause1, clause2):
    """Resolve two clauses and return the resolvents."""
    resolvents = []
    for predicate1 in clause1:
        for predicate2 in clause2:
            theta = unify(predicate1, negate(predicate2))
            if theta is not None:
                new_clause = (substitute(clause1, theta) | substitute(clause2, theta)) - {predicate1, predicate2}
                resolvents.append(frozenset(new_clause))
    return resolvents

def negate(predicate):
    """Negate a predicate."""
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def substitute(clause, theta):
    """Apply substitutions to a clause."""
    return {substitute_predicate(p, theta) for p in clause}

def substitute_predicate(predicate, theta):
    """Apply substitution to a single predicate."""
    if isinstance(predicate, str):
        return theta.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(theta.get(arg, arg) for arg in predicate[1:])

def resolution(kb, query):
    """Perform resolution to prove the query."""
    negated_query = frozenset({negate(query)})
    clauses = kb | {negated_query}
    new_clauses = set()

    while True:
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = resolve(clause1, clause2)
            if frozenset() in resolvents:
                return True
            new_clauses.update(resolvents)
        if new_clauses.issubset(clauses):
            return False
        clauses |= new_clauses

# Knowledge Base (in CNF)
kb = {
    frozenset({('Mother', 'leela', 'Oshin')}),  # Leela is mother of Oshin
    frozenset({('Alive', 'leela')}),                     # Leela is alive
    frozenset({('not', ('Mother','x','y')),('Parent','x','y')}),    # Every Mother will be a parent
    frozenset({('not',('Parent','x','y')),('not',('Alive','x')),('Older','x','y')}),  # If a parent is alive then the parent is older than its child
}

# Query: Prove Leela is older than Oshin
query = ('Older', 'leela', 'Oshin')

# Run resolution
result = resolution(kb, query)
if result:
    print("Proven: Leela is older than Oshin.")
else:
    print("Cannot prove: Leela is older than Oshin.")

print("\n\nAditya Dinesh Netrakar")
print("USN: 1BM22CS017")

Proven: Leela is older than Oshin.


Aditya Dinesh Netrakar
USN: 1BM22CS017
